In [1]:
import pandas as pd
import pymysql
import os
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
# Configuración BD
DB_HOST = os.getenv("IP") 
DB_USER = "rrhh_master" 
DB_PASSWORD = os.getenv("clave_sql") 
DB_NAME = "rrhh_app" 

conexion = pymysql.connect(
    host=DB_HOST,
    user=DB_USER,
    password=DB_PASSWORD,
    charset='utf8mb4'
)
cursor = conexion.cursor()

In [3]:
sql_query = f"""
SELECT
    e.full_name,
    e.rut,
    e.active_since,
    e.status,
    e.name_role,
    e.cost_center,
    a.first_level_name AS area_name
FROM
    rrhh_app.employees AS e
INNER JOIN
    rrhh_app.areas AS a ON e.area_id = a.id
WHERE
    e.name_role IN (
        'Analista de Calidad y Procesos',
        'Asistente De Servicios Generales',
        'Analista de Control De Calidad',
        'Analista De Microbiología',
        'Asistente De Laboratorio',
        'Coordinadora Muestras',
        'Inspector De Proceso',
        'Administrativo Bodega Despacho',
        'Ayudante De Bodega',
        'Asistente De Bodega',
        'Chofer Administrativo Transporte',
        'Chofer',
        'Coordinador de Planta',
        'Encargado De Bodega Inflamables',
        'Encargado De Bodega Materias Primas',
        'Operario',
        'Operario Almacenamiento y Gestión de Residuos',
        'Peoneta'
    )
    AND e.status = 'activo'
ORDER BY
    e.active_since DESC;
"""

#1 Importar dataframes

In [4]:
from IPython.display import display
# Utiliza pandas.read_sql para ejecutar la consulta con la conexión ya abierta
df_resultado = pd.read_sql(sql_query, conexion)

print("\n--- ¡Consulta ejecutada y DataFrame creado! ---")
#display(df_resultado.head())

print(f"\nNúmero de filas y columnas: {df_resultado.shape}")
print("Columnas y tipos de datos:")
df_resultado.info()
display(df_resultado.head())


--- ¡Consulta ejecutada y DataFrame creado! ---

Número de filas y columnas: (282, 7)
Columnas y tipos de datos:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 282 entries, 0 to 281
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   full_name     282 non-null    object
 1   rut           282 non-null    object
 2   active_since  282 non-null    object
 3   status        282 non-null    object
 4   name_role     282 non-null    object
 5   cost_center   282 non-null    object
 6   area_name     282 non-null    object
dtypes: object(7)
memory usage: 15.6+ KB


C:\Users\bgacitua\AppData\Local\Temp\ipykernel_4648\3548020417.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_resultado = pd.read_sql(sql_query, conexion)


,full_name,rut,active_since,status,name_role,cost_center,area_name
0,Víctor Antonio Riquelme Ulloa,14.090.215-2,2025-09-08,activo,Operario,6005,CARLOS CRAMER PRODUCTOS AROMÁTICOS S.A. C.I.
1,Luis Andrés Rojas Romero,16.459.775-k,2025-09-08,activo,Operario,6005,CARLOS CRAMER PRODUCTOS AROMÁTICOS S.A. C.I.
2,Araceli Alejandra Berríos Vera,19.916.324-8,2025-09-08,activo,Inspector De Proceso,6106,CARLOS CRAMER PRODUCTOS AROMÁTICOS S.A. C.I.
3,Kevin Hernán Zamora Flores,20.419.421-1,2025-09-08,activo,Operario,6005,CARLOS CRAMER PRODUCTOS AROMÁTICOS S.A. C.I.
4,Fernando Alonso Vera Clavero,22.193.159-9,2025-09-01,activo,Peoneta,6205,Servicios De Producción Y Logística Ccpa Ltda.


In [5]:
#Importar dataframe de incidencias con la condición de fechas y periodos. 

sql_query_incidencias = f"""
SELECT DISTINCT
    ci.*, 
    e.full_name,
    e.name_role,
    e.rut,
    CASE
        WHEN ci.start_date >= '2024-12-07' AND ci.start_date <= '2025-01-06' THEN 'Enero'
        WHEN ci.start_date >= '2025-01-07' AND ci.start_date <= '2025-02-05' THEN 'Febrero'
        WHEN ci.start_date >= '2025-02-06' AND ci.start_date <= '2025-03-06' THEN 'Marzo'
        WHEN ci.start_date >= '2025-03-07' AND ci.start_date <= '2025-04-06' THEN 'Abril'
        WHEN ci.start_date >= '2025-04-07' AND ci.start_date <= '2025-05-06' THEN 'Mayo'
        WHEN ci.start_date >= '2025-05-07' AND ci.start_date <= '2025-06-05' THEN 'Junio'
        WHEN ci.start_date >= '2025-06-06' AND ci.start_date <= '2025-07-05' THEN 'Julio'
        WHEN ci.start_date >= '2025-07-06' AND ci.start_date <= '2025-08-05' THEN 'Agosto'
        WHEN ci.start_date >= '2025-08-06' AND ci.start_date <= '2025-09-04' THEN 'Septiembre'
        WHEN ci.start_date >= '2025-09-05' AND ci.start_date <= '2025-10-04' THEN 'Octubre'
        WHEN ci.start_date >= '2025-10-05' AND ci.start_date <= '2025-11-03' THEN 'Noviembre'
        WHEN ci.start_date >= '2025-11-04' AND ci.start_date <= '2025-12-03' THEN 'Diciembre'
        WHEN ci.start_date >= '2025-12-04' AND ci.start_date <= '2026-01-03' THEN 'Enero'
        ELSE 'Año anterior'
    END AS mes,
    CASE
        WHEN ci.start_date >= '2024-12-07' AND ci.start_date <= '2024-12-22' THEN '1 Quincena'
        WHEN ci.start_date >= '2024-12-23' AND ci.start_date <= '2025-01-06' THEN '2 Quincena'
        WHEN ci.start_date >= '2025-01-07' AND ci.start_date <= '2025-01-21' THEN '1 Quincena'
        WHEN ci.start_date >= '2025-01-22' AND ci.start_date <= '2025-02-05' THEN '2 Quincena'
        WHEN ci.start_date >= '2025-02-06' AND ci.start_date <= '2025-02-20' THEN '1 Quincena'
        WHEN ci.start_date >= '2025-02-21' AND ci.start_date <= '2025-03-06' THEN '2 Quincena'
        WHEN ci.start_date >= '2025-03-07' AND ci.start_date <= '2025-03-21' THEN '1 Quincena'
        WHEN ci.start_date >= '2025-03-22' AND ci.start_date <= '2025-04-06' THEN '2 Quincena'
        WHEN ci.start_date >= '2025-04-07' AND ci.start_date <= '2025-04-21' THEN '1 Quincena'
        WHEN ci.start_date >= '2025-04-22' AND ci.start_date <= '2025-05-06' THEN '2 Quincena'
        WHEN ci.start_date >= '2025-05-07' AND ci.start_date <= '2025-05-21' THEN '1 Quincena'
        WHEN ci.start_date >= '2025-05-22' AND ci.start_date <= '2025-06-05' THEN '2 Quincena'
        WHEN ci.start_date >= '2025-06-06' AND ci.start_date <= '2025-06-20' THEN '1 Quincena'
        WHEN ci.start_date >= '2025-06-21' AND ci.start_date <= '2025-07-05' THEN '2 Quincena'
        WHEN ci.start_date >= '2025-07-06' AND ci.start_date <= '2025-07-20' THEN '1 Quincena'
        WHEN ci.start_date >= '2025-07-21' AND ci.start_date <= '2025-08-05' THEN '2 Quincena'
        WHEN ci.start_date >= '2025-08-06' AND ci.start_date <= '2025-08-20' THEN '1 Quincena'
        WHEN ci.start_date >= '2025-08-21' AND ci.start_date <= '2025-09-04' THEN '2 Quincena'
        WHEN ci.start_date >= '2025-09-05' AND ci.start_date <= '2025-09-19' THEN '1 Quincena'
        WHEN ci.start_date >= '2025-09-20' AND ci.start_date <= '2025-10-04' THEN '2 Quincena'
        WHEN ci.start_date >= '2025-10-05' AND ci.start_date <= '2025-10-19' THEN '1 Quincena'
        WHEN ci.start_date >= '2025-10-20' AND ci.start_date <= '2025-11-03' THEN '2 Quincena'
        WHEN ci.start_date >= '2025-11-04' AND ci.start_date <= '2025-11-18' THEN '1 Quincena'
        WHEN ci.start_date >= '2025-11-19' AND ci.start_date <= '2025-12-03' THEN '2 Quincena'
        WHEN ci.start_date >= '2025-12-04' AND ci.start_date <= '2025-12-18' THEN '1 Quincena'
        WHEN ci.start_date >= '2025-12-19' AND ci.start_date <= '2026-01-03' THEN '2 Quincena'
        ELSE 'Año anterior'
    END AS quincena
FROM 
    rrhh_app.consolidado_incidencias ci
INNER JOIN 
    rrhh_app.employees e ON ci.employee_id = e.id
WHERE ci.start_date > '2025-08-05' and ci.start_date < '2025-09-04'
AND e.name_role IN (
'Analista de Calidad y Procesos',
'Asistente De Servicios Generales', 
'Analista de Control De Calidad', 
'Analista De Microbiología', 
'Asistente De Laboratorio', 
'Coordinadora Muestras', 
'Inspector De Proceso', 
'Administrativo Bodega Despacho', 
'Ayudante De Bodega', 
'Asistente De Bodega', 
'Chofer Administrativo Transporte', 
'Chofer', 
'Coordinador de Planta', 
'Encargado De Bodega Inflamables', 
'Encargado De Bodega Materias Primas', 
'Operario', 
'Operario Almacenamiento y Gestión de Residuos', 
'Peoneta');
"""

In [6]:
# Utiliza pandas.read_sql para ejecutar la consulta con la conexión ya abierta
df_incidencias = pd.read_sql(sql_query_incidencias, conexion)

print("\n--- ¡Consulta ejecutada y DataFrame creado! ---")
#display(df_resultado.head())

print(f"\nNúmero de filas y columnas: {df_incidencias.shape}")
print("Columnas y tipos de datos:")
df_incidencias.info()
display(df_incidencias.head())


--- ¡Consulta ejecutada y DataFrame creado! ---

Número de filas y columnas: (26, 16)
Columnas y tipos de datos:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26 entries, 0 to 25
Data columns (total 16 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   id                    26 non-null     object
 1   start_date            26 non-null     object
 2   end_date              26 non-null     object
 3   days_count            26 non-null     object
 4   workday_stage         26 non-null     object
 5   application_date      26 non-null     object
 6   application_end_date  26 non-null     object
 7   employee_id           26 non-null     object
 8   status                26 non-null     object
 9   created_at            26 non-null     object
 10  tabla_origen          26 non-null     object
 11  full_name             26 non-null     object
 12  name_role             26 non-null     object
 13  rut                   26 non

C:\Users\bgacitua\AppData\Local\Temp\ipykernel_4648\1838538706.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_incidencias = pd.read_sql(sql_query_incidencias, conexion)


,id,start_date,end_date,days_count,workday_stage,application_date,application_end_date,employee_id,status,created_at,tabla_origen,full_name,name_role,rut,mes,quincena
0,109144,2025-09-02,2025-09-12,11,full_working_day,2025-09-02,2025-09-12,4231,approved,2025-09-02T15:35:57.881-04:00,licences,Richard Agustin Bustamante Castro,Operario,12.314.132-6,Septiembre,2 Quincena
1,108253,2025-08-11,2025-09-07,28,full_working_day,2025-08-11,2025-09-07,4286,approved,2025-08-12T09:08:20.605-04:00,licences,Marcelo Nicanor Morales Barrera,Operario,13.558.836-9,Septiembre,1 Quincena
2,109244,2025-08-29,2025-08-29,1,full_working_day,2025-08-29,2025-08-29,4304,approved,2025-09-03T11:19:17.252-04:00,permissions,Joel Esteban Roman Gonzalez,Operario,13.935.403-6,Septiembre,2 Quincena
3,109078,2025-09-02,2025-09-02,0.18292683,full_working_day,2025-09-02,2025-09-02,11587,approved,2025-09-01T16:58:54.932-04:00,permissions,Marcelo Del Carmen González Baeza,Operario,14.485.044-0,Septiembre,2 Quincena
4,108946,2025-08-27,2025-08-27,1,start_working_day,2025-08-27,2025-08-27,4319,approved,2025-08-28T08:12:22.638-04:00,permissions,Cesar Antonio Solis Cartes,Asistente De Laboratorio,15.172.551-1,Septiembre,2 Quincena


In [8]:
#Importa dataframe de marcas corregidas
df_marcas = pd.read_excel(r"C:\Users\bgacitua\Desktop\Repositorio_GitHub\Scripts de Python\Script Bono Asistencia\Archivos de trabajo\reporte_turnos_marcas_corregidas_20250910164646.xlsx")
display(df_marcas.head())
df_marcas.info()

,Rut,Nombre,Unidad,Puesto,Fecha,Tipo de turno,Entrada programada,Salida programada,Marca de entrada,Marca de salida,Motivo marca de entrada,Comentario marca de entrada,Motivo marca de salida,Comentario marca de salida,Tipo de corrección
0,6986640-9,Gloria Emilia Gonzalez Castro,Administación y Lavandería,Secretaria,06-08-2025,T,07:50,17:00,07:50:00 *,17:00:00 *,Olvido de Marcaje,NaN,Olvido de Marcaje,NaN,Entrada y salida
1,6986640-9,Gloria Emilia Gonzalez Castro,Administación y Lavandería,Secretaria,08-08-2025,D,07:50,15:30,07:30:44,15:30:00 *,NaN,NaN,Olvido de Marcaje,Olvido marca,Solo salida
2,6986640-9,Gloria Emilia Gonzalez Castro,Administación y Lavandería,Secretaria,12-08-2025,T,07:50,17:00,07:50:00 *,16:55:48,Olvido de Marcaje,NaN,NaN,NaN,Solo entrada
3,6986640-9,Gloria Emilia Gonzalez Castro,Administación y Lavandería,Secretaria,28-08-2025,T,07:50,17:00,07:03:12,17:00:00 *,NaN,NaN,Olvido de Marcaje,Olvido marca,Solo salida
4,19243690-7,María Francisca Ulloa Valdés,Desarrollo Sabores General,Analista,06-08-2025,T,07:50,17:00,07:36:28,17:21:00,NaN,NaN,NaN,NaN,Entrada y salida


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1013 entries, 0 to 1012
Data columns (total 15 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   Rut                          1013 non-null   object
 1   Nombre                       1013 non-null   object
 2   Unidad                       1013 non-null   object
 3   Puesto                       1013 non-null   object
 4   Fecha                        1013 non-null   object
 5   Tipo de turno                1013 non-null   object
 6   Entrada programada           1013 non-null   object
 7   Salida programada            1013 non-null   object
 8   Marca de entrada             1013 non-null   object
 9   Marca de salida              1013 non-null   object
 10  Motivo marca de entrada      556 non-null    object
 11  Comentario marca de entrada  530 non-null    object
 12  Motivo marca de salida       755 non-null    object
 13  Comentario marca de salida   711 

In [9]:
#Importar dataframe atrasos
columnas_necesarias = ['nombreTipoAusencia' ,'fecha', 'horaInicio', 'horaProgramadaInicio', 'HoraInicioMarcaReloj', 'horaTermino', 'horaProgramadaTermino', 'HoraTerminoMarcaReloj', 'rut', 'nombre', 'apellidoPaterno', 'apellidoMaterno', 'Centrocosto_trabajador', 'PuestoIntegracion_trabajador', 'nombreMotivoPago']
df_atrasos = pd.read_excel(r"C:\Users\bgacitua\Desktop\Repositorio_GitHub\Scripts de Python\Script Bono Asistencia\Archivos de trabajo\2025-09-12 08_30_21-2018778312-reporte_turnos_bruto_combinado.xlsx")
df_atrasos = df_atrasos[columnas_necesarias]
df_atrasos.info()

# -- Crear dataframe con las incidencias de los empleados. Puede ser que haya discrepancias con BUK -- #


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10730 entries, 0 to 10729
Data columns (total 15 columns):
 #   Column                        Non-Null Count  Dtype 
---  ------                        --------------  ----- 
 0   nombreTipoAusencia            10730 non-null  object
 1   fecha                         10730 non-null  object
 2   horaInicio                    10730 non-null  object
 3   horaProgramadaInicio          10730 non-null  object
 4   HoraInicioMarcaReloj          10730 non-null  object
 5   horaTermino                   10730 non-null  object
 6   horaProgramadaTermino         10730 non-null  object
 7   HoraTerminoMarcaReloj         10730 non-null  object
 8   rut                           10730 non-null  object
 9   nombre                        10730 non-null  object
 10  apellidoPaterno               10730 non-null  object
 11  apellidoMaterno               10730 non-null  object
 12  Centrocosto_trabajador        10730 non-null  object
 13  PuestoIntegracio

In [10]:
display(df_atrasos.head())

,nombreTipoAusencia,fecha,horaInicio,horaProgramadaInicio,HoraInicioMarcaReloj,horaTermino,horaProgramadaTermino,HoraTerminoMarcaReloj,rut,nombre,apellidoPaterno,apellidoMaterno,Centrocosto_trabajador,PuestoIntegracion_trabajador,nombreMotivoPago
0,,2025-08-06,07:50:00,2025-08-06 07:50:00,2025-08-06 07:50:00,17:00:00,2025-08-06 17:00:00,2025-08-06 17:00:00,69866409,Gloria,Gonzalez,Castro,Administración,Secretaria,
1,,2025-08-07,07:50:00,2025-08-07 07:50:00,2025-08-07 07:02:01,17:00:00,2025-08-07 17:00:00,2025-08-07 16:57:16,69866409,Gloria,Gonzalez,Castro,Administración,Secretaria,
2,,2025-08-08,07:50:00,2025-08-08 07:50:00,2025-08-08 07:30:44,15:30:00,2025-08-08 15:30:00,2025-08-08 15:30:00,69866409,Gloria,Gonzalez,Castro,Administración,Secretaria,
3,,2025-08-11,07:50:00,2025-08-11 07:50:00,2025-08-11 07:23:45,17:00:00,2025-08-11 17:00:00,2025-08-11 17:01:52,69866409,Gloria,Gonzalez,Castro,Administración,Secretaria,
4,,2025-08-12,07:50:00,2025-08-12 07:50:00,2025-08-12 07:50:00,17:00:00,2025-08-12 17:00:00,2025-08-12 16:55:48,69866409,Gloria,Gonzalez,Castro,Administración,Secretaria,


#2 Exploración y limpieza de dataframes

In [11]:
display(df_marcas.head())
print(f"Número de filas: {df_marcas.shape[0]}")

,Rut,Nombre,Unidad,Puesto,Fecha,Tipo de turno,Entrada programada,Salida programada,Marca de entrada,Marca de salida,Motivo marca de entrada,Comentario marca de entrada,Motivo marca de salida,Comentario marca de salida,Tipo de corrección
0,6986640-9,Gloria Emilia Gonzalez Castro,Administación y Lavandería,Secretaria,06-08-2025,T,07:50,17:00,07:50:00 *,17:00:00 *,Olvido de Marcaje,NaN,Olvido de Marcaje,NaN,Entrada y salida
1,6986640-9,Gloria Emilia Gonzalez Castro,Administación y Lavandería,Secretaria,08-08-2025,D,07:50,15:30,07:30:44,15:30:00 *,NaN,NaN,Olvido de Marcaje,Olvido marca,Solo salida
2,6986640-9,Gloria Emilia Gonzalez Castro,Administación y Lavandería,Secretaria,12-08-2025,T,07:50,17:00,07:50:00 *,16:55:48,Olvido de Marcaje,NaN,NaN,NaN,Solo entrada
3,6986640-9,Gloria Emilia Gonzalez Castro,Administación y Lavandería,Secretaria,28-08-2025,T,07:50,17:00,07:03:12,17:00:00 *,NaN,NaN,Olvido de Marcaje,Olvido marca,Solo salida
4,19243690-7,María Francisca Ulloa Valdés,Desarrollo Sabores General,Analista,06-08-2025,T,07:50,17:00,07:36:28,17:21:00,NaN,NaN,NaN,NaN,Entrada y salida


Número de filas: 1013


In [12]:
#Eliminar filas con 'Falla Reloj' y 'Otro' en ambas columnas
df_marcas_filtrado = df_marcas[
    ~df_marcas['Motivo marca de entrada'].isin(['Falla Reloj', 'Otro']) & 
    ~df_marcas['Motivo marca de salida'].isin(['Falla Reloj', 'Otro'])
]

print(f"Filas originales: {df_marcas.shape[0]}")
print(f"Filas después del filtro: {df_marcas_filtrado.shape[0]}")
display(df_marcas_filtrado.head())

Filas originales: 1013
Filas después del filtro: 634


,Rut,Nombre,Unidad,Puesto,Fecha,Tipo de turno,Entrada programada,Salida programada,Marca de entrada,Marca de salida,Motivo marca de entrada,Comentario marca de entrada,Motivo marca de salida,Comentario marca de salida,Tipo de corrección
0,6986640-9,Gloria Emilia Gonzalez Castro,Administación y Lavandería,Secretaria,06-08-2025,T,07:50,17:00,07:50:00 *,17:00:00 *,Olvido de Marcaje,NaN,Olvido de Marcaje,NaN,Entrada y salida
1,6986640-9,Gloria Emilia Gonzalez Castro,Administación y Lavandería,Secretaria,08-08-2025,D,07:50,15:30,07:30:44,15:30:00 *,NaN,NaN,Olvido de Marcaje,Olvido marca,Solo salida
2,6986640-9,Gloria Emilia Gonzalez Castro,Administación y Lavandería,Secretaria,12-08-2025,T,07:50,17:00,07:50:00 *,16:55:48,Olvido de Marcaje,NaN,NaN,NaN,Solo entrada
3,6986640-9,Gloria Emilia Gonzalez Castro,Administación y Lavandería,Secretaria,28-08-2025,T,07:50,17:00,07:03:12,17:00:00 *,NaN,NaN,Olvido de Marcaje,Olvido marca,Solo salida
4,19243690-7,María Francisca Ulloa Valdés,Desarrollo Sabores General,Analista,06-08-2025,T,07:50,17:00,07:36:28,17:21:00,NaN,NaN,NaN,NaN,Entrada y salida


In [13]:
#Línea para revisar que solo hayan "olvidos de marca"
display(df_marcas_filtrado['Motivo marca de salida'].value_counts())

Motivo marca de salida
Olvido de Marcaje    390
Name: count, dtype: int64

In [14]:
## -- Limpiar df_atrasos -- ##

#Limpieza del formato de las columnas con datetime para dejar solo la hora 
columnas_a_limpiar = ['horaProgramadaInicio', 'HoraInicioMarcaReloj', 'horaProgramadaTermino', 'HoraTerminoMarcaReloj']

for columna in columnas_a_limpiar:
    if columna in df_atrasos.columns:
        # Dividir por espacio y tomar la parte después del primer espacio
        df_atrasos[columna] = df_atrasos[columna].str.split(' ', n=1, expand=True)[1]

df_atrasos['nombreCompleto'] = df_atrasos['nombre'].str.cat([
    df_atrasos['apellidoPaterno'], 
    df_atrasos['apellidoMaterno']], sep=' ', na_rep='')

#Verificar los cambios
#display(df_atrasos.head())

#Filtrar por los puestos sujetos a bono asistencia
puestos_filtrados = [
'Analista de Calidad y Procesos',
'Asistente De Servicios Generales', 
'Analista de Control De Calidad', 
'Analista De Microbiología', 
'Asistente De Laboratorio', 
'Coordinadora Muestras', 
'Inspector De Proceso', 
'Administrativo Bodega Despacho', 
'Ayudante De Bodega', 
'Asistente De Bodega', 
'Chofer Administrativo Transporte', 
'Chofer', 
'Coordinador de Planta', 
'Encargado De Bodega Inflamables', 
'Encargado De Bodega Materias Primas', 
'Operario', 
'Operario Almacenamiento y Gestión de Residuos', 
'Peoneta']

#Mostrar dataframe filtrado de las personas con puestos sujetos a bono asistencia
df_atrasos_filtrado = df_atrasos[df_atrasos['PuestoIntegracion_trabajador'].isin(puestos_filtrados)]
print(f"Filas originales: {df_atrasos.shape[0]}")
print(f"Filas después del filtro: {df_atrasos_filtrado.shape[0]}")
display(df_atrasos_filtrado.head())

Filas originales: 10730
Filas después del filtro: 5644


,nombreTipoAusencia,fecha,horaInicio,horaProgramadaInicio,HoraInicioMarcaReloj,horaTermino,horaProgramadaTermino,HoraTerminoMarcaReloj,rut,nombre,apellidoPaterno,apellidoMaterno,Centrocosto_trabajador,PuestoIntegracion_trabajador,nombreMotivoPago,nombreCompleto
7,,2025-08-06,07:50:00,07:50:00,07:20:36,17:00:00,17:00:00,16:55:11,128294988,Cesar,Vera,Ibarra,Administración,Asistente De Servicios Generales,,Cesar Vera Ibarra
8,,2025-08-07,07:50:00,07:50:00,07:18:57,17:00:00,17:00:00,16:52:30,128294988,Cesar,Vera,Ibarra,Administración,Asistente De Servicios Generales,,Cesar Vera Ibarra
9,,2025-08-08,07:50:00,07:50:00,07:27:16,15:30:00,15:30:00,15:30:00,128294988,Cesar,Vera,Ibarra,Administración,Asistente De Servicios Generales,,Cesar Vera Ibarra
10,,2025-08-11,07:50:00,07:50:00,07:19:18,17:00:00,17:00:00,16:53:47,128294988,Cesar,Vera,Ibarra,Administración,Asistente De Servicios Generales,,Cesar Vera Ibarra
11,,2025-08-12,07:50:00,07:50:00,07:15:15,17:00:00,17:00:00,16:53:18,128294988,Cesar,Vera,Ibarra,Administración,Asistente De Servicios Generales,,Cesar Vera Ibarra


In [15]:
df_incidencias.columns

Index(['id', 'start_date', 'end_date', 'days_count', 'workday_stage',
       'application_date', 'application_end_date', 'employee_id', 'status',
       'created_at', 'tabla_origen', 'full_name', 'name_role', 'rut', 'mes',
       'quincena'],
      dtype='object')

#3 Procesamiento de dataframes

In [16]:
#Asignar periodo de fecha a los atrasos
import pandas as pd

fecha_inicio_q1 = pd.Timestamp('2025-08-06')
fecha_fin_q1 = pd.Timestamp('2025-08-20')
fecha_inicio_q2 = pd.Timestamp('2025-08-21')
fecha_fin_q2 = pd.Timestamp('2025-09-04')

#Agrupamiento y resumen de permisos, licencias e inasistencias
#Este agrupamiento sirve únicamente para el formato pedido por Juan Cárcamo

df_incidencias['start_date'] = pd.to_datetime(df_incidencias['start_date'], errors='coerce')

df_incidencias['periodo'] = df_incidencias['start_date'].apply(
    lambda x: 'Periodo 1' if fecha_inicio_q1 <= x <= fecha_fin_q1 else (
        'Periodo 2' if fecha_inicio_q2 <= x <= fecha_fin_q2 else 'Fuera de rango'
    )
)

resumen_incidencias = df_incidencias.groupby(['rut', 'full_name', 'tabla_origen']).agg({
    'tabla_origen': 'count',  # Contar total de incidencias
    'start_date': ['min', 'max']  # Fecha primera y última incidencia
}).reset_index()

# Aplanar las columnas
resumen_incidencias.columns = ['rut', 'full_name', 'tabla_origen', 'total_incidencias', 'primera_incidencia', 'ultima_incidencia']

print(f'Cantidad de personas encontrados = {resumen_incidencias.shape[0]}')
display(resumen_incidencias)

Cantidad de personas encontrados = 23


,rut,full_name,tabla_origen,total_incidencias,primera_incidencia,ultima_incidencia
0,12.314.132-6,Richard Agustin Bustamante Castro,licences,1,2025-09-02,2025-09-02
1,13.558.836-9,Marcelo Nicanor Morales Barrera,licences,1,2025-08-11,2025-08-11
2,13.935.403-6,Joel Esteban Roman Gonzalez,permissions,1,2025-08-29,2025-08-29
3,14.485.044-0,Marcelo Del Carmen González Baeza,permissions,1,2025-09-02,2025-09-02
4,15.172.551-1,Cesar Antonio Solis Cartes,permissions,2,2025-08-25,2025-08-27
5,15.820.779-6,Álvaro José Astorga Hinostroza,absences,1,2025-09-02,2025-09-02
6,15.820.779-6,Álvaro José Astorga Hinostroza,permissions,1,2025-09-01,2025-09-01
7,15.934.416-9,Alexander Araos Deramond,absences,1,2025-08-29,2025-08-29
8,16.323.864-0,Francisco Javier Vásquez Adrián,licences,1,2025-08-23,2025-08-23
9,17.049.574-8,Patricio Andrés Gutiérrez Pérez,licences,1,2025-08-10,2025-08-10


In [17]:
#Procesamiento Df_atrasos_filtrado ---> Reporte de rflex (Bruto combinado)
#Columnas df_atrasos_filtrado
'''
columnas_df_atrasos_filt = df_atrasos_filtrado.columns.tolist()
print(columnas_df_atrasos_filt)

tipos_ausencias = df_atrasos_filtrado['nombreTipoAusencia'].unique()
print(tipos_ausencias)
'''
#Eliminar filas vacías
df_atrasos_filtrado_permisos = df_atrasos_filtrado.copy()

# Eliminar filas donde 'nombreTipoAusencia' es None, NaN, vacío o solo espacios
df_atrasos_filtrado_permisos = df_atrasos_filtrado_permisos[
    df_atrasos_filtrado_permisos['nombreTipoAusencia'].notna() &
    (df_atrasos_filtrado_permisos['nombreTipoAusencia'].str.strip() != '')
]

#Revisión de eliminación de filas sin permisos. 
print(df_atrasos_filtrado_permisos['nombreTipoAusencia'].unique())

['Vacación' 'Licencia postnatal' 'Licencia Accidente Trayecto'
 'Permiso sin goce de sueldo' 'Permiso con goce de sueldo'
 'Vacación Progresiva' 'Inasistencia' 'Licencia' 'Permiso por matrimonio'
 'Licencia Accidente Trabajo' 'Licencia Accidente' 'Capacitacion']


In [18]:
df_atrasos_filtrado_permisos.columns

Index(['nombreTipoAusencia', 'fecha', 'horaInicio', 'horaProgramadaInicio',
       'HoraInicioMarcaReloj', 'horaTermino', 'horaProgramadaTermino',
       'HoraTerminoMarcaReloj', 'rut', 'nombre', 'apellidoPaterno',
       'apellidoMaterno', 'Centrocosto_trabajador',
       'PuestoIntegracion_trabajador', 'nombreMotivoPago', 'nombreCompleto'],
      dtype='object')

In [19]:
#Agrupar datos encontrados en Reporte Rflex Bruto Combinado

permisos_descuentan = ['Inasistencia', 'Licencia', 'Licencia Accidente', 'Licencia Accidente Trabajo', 
                    'Licencia Accidente Trayecto', 'Licencia postnatal', 'Permiso con goce de sueldo', 
                    'Permiso sin goce de sueldo']

df_atrasos_filtrado_permisos = df_atrasos_filtrado_permisos[df_atrasos_filtrado_permisos['nombreTipoAusencia'].isin(permisos_descuentan)]

df_atrasos_filtrado_permisos['fecha'] = pd.to_datetime(df_atrasos_filtrado_permisos['fecha'], errors='coerce')

df_atrasos_filtrado_permisos['periodo'] = df_atrasos_filtrado_permisos['fecha'].apply(
    lambda x: 'Periodo 1' if fecha_inicio_q1 <= x <= fecha_fin_q1 else (
        'Periodo 2' if fecha_inicio_q2 <= x <= fecha_fin_q2 else 'Fuera de rango'
    )
)

resumen_atrasos_filtrado_permisos = df_atrasos_filtrado_permisos.groupby(
    ['nombreTipoAusencia', 'rut', 'nombreCompleto', 'periodo', 
    'Centrocosto_trabajador','PuestoIntegracion_trabajador']).agg(
        {
    'nombreCompleto': 'count',
    'fecha': ['min', 'max']
})


# Mostrar hasta 100 filas (puedes ajustar el número si necesitas más)
#pd.set_option('display.max_rows', 100)

#---------------------------------------------------------------------------------------------------------------------#
###El reporte de rflex entrega información más detallada acerca de los permisos y tipos de permiso ###
###en comparación con las incidencias almacenadas en la base de buk. -----> Trabajar en un mayor detalle en la BD ###
#---------------------------------------------------------------------------------------------------------------------#

display(resumen_atrasos_filtrado_permisos.head())

nombreCompleto  \
                                                                                                                                             count   
nombreTipoAusencia rut       nombreCompleto            periodo   Centrocosto_trabajador                PuestoIntegracion_trabajador                  
Inasistencia       158207796 Álvaro Astorga Hinostroza Periodo 2 Secador Spray                         Operario                                  1   
                   159344169 Alexander Araos Deramond  Periodo 2 Fabricación Y Envasado Mezclas Polvos Operario                                  1   
                   177812021 Matías Muñoz Lagos        Periodo 1 Fabricación Y Envasado Mezclas Polvos Operario                                  1   
                                                       Periodo 2 Fabricación Y Envasado Mezclas Polvos Operario                                  1   
                   187654173 Diego Islas Huenchuman    Periodo 1 Despacho                              Ayudante De Bodega                        1   

                                                                                                                                         fecha  \
                                                                                                                                           min   
nombreTipoAusencia rut       nombreCompleto            periodo   Centrocosto_trabajador                PuestoIntegracion_trabajador              
Inasistencia       158207796 Álvaro Astorga Hinostroza Periodo 2 Secador Spray                         Operario                     2025-09-02   
                   159344169 Alexander Araos Deramond  Periodo 2 Fabricación Y Envasado Mezclas Polvos Operario                     2025-08-29   
                   177812021 Matías Muñoz Lagos        Periodo 1 Fabricación Y Envasado Mezclas Polvos Operario                     2025-08-13   
                                                       Periodo 2 Fabricación Y Envasado Mezclas Polvos Operario                     2025-08-28   
                   187654173 Diego Islas Huenchuman    Periodo 1 Despacho                              Ayudante De Bodega           2025-08-08   

                                                                                                                                                
                                                                                                                                           max  
nombreTipoAusencia rut       nombreCompleto            periodo   Centrocosto_trabajador                PuestoIntegracion_trabajador             
Inasistencia       158207796 Álvaro Astorga Hinostroza Periodo 2 Secador Spray                         Operario                     2025-09-02  
                   159344169 Alexander Araos Deramond  Periodo 2 Fabricación Y Envasado Mezclas Polvos Operario                     2025-08-29  
                   177812021 Matías Muñoz Lagos        Periodo 1 Fabricación Y Envasado Mezclas Polvos Operario                     2025-08-13  
                                                       Periodo 2 Fabricación Y Envasado Mezclas Polvos Operario                     2025-08-28  
                   187654173 Diego Islas Huenchuman    Periodo 1 Despacho                              Ayudante De Bodega           2025-08-08

In [20]:
#Revisión de eliminación de permisos que no descuentan bono.
print(df_atrasos_filtrado_permisos['nombreTipoAusencia'].unique())

['Licencia postnatal' 'Licencia Accidente Trayecto'
 'Permiso sin goce de sueldo' 'Permiso con goce de sueldo' 'Inasistencia'
 'Licencia' 'Licencia Accidente Trabajo' 'Licencia Accidente']


In [21]:
df_marcas_filtrado.columns

Index(['Rut', 'Nombre', 'Unidad', 'Puesto', 'Fecha', 'Tipo de turno',
       'Entrada programada', 'Salida programada', 'Marca de entrada',
       'Marca de salida', 'Motivo marca de entrada',
       'Comentario marca de entrada', 'Motivo marca de salida',
       'Comentario marca de salida', 'Tipo de corrección'],
      dtype='object')

In [22]:
display(df_marcas_filtrado.head())

,Rut,Nombre,Unidad,Puesto,Fecha,Tipo de turno,Entrada programada,Salida programada,Marca de entrada,Marca de salida,Motivo marca de entrada,Comentario marca de entrada,Motivo marca de salida,Comentario marca de salida,Tipo de corrección
0,6986640-9,Gloria Emilia Gonzalez Castro,Administación y Lavandería,Secretaria,06-08-2025,T,07:50,17:00,07:50:00 *,17:00:00 *,Olvido de Marcaje,NaN,Olvido de Marcaje,NaN,Entrada y salida
1,6986640-9,Gloria Emilia Gonzalez Castro,Administación y Lavandería,Secretaria,08-08-2025,D,07:50,15:30,07:30:44,15:30:00 *,NaN,NaN,Olvido de Marcaje,Olvido marca,Solo salida
2,6986640-9,Gloria Emilia Gonzalez Castro,Administación y Lavandería,Secretaria,12-08-2025,T,07:50,17:00,07:50:00 *,16:55:48,Olvido de Marcaje,NaN,NaN,NaN,Solo entrada
3,6986640-9,Gloria Emilia Gonzalez Castro,Administación y Lavandería,Secretaria,28-08-2025,T,07:50,17:00,07:03:12,17:00:00 *,NaN,NaN,Olvido de Marcaje,Olvido marca,Solo salida
4,19243690-7,María Francisca Ulloa Valdés,Desarrollo Sabores General,Analista,06-08-2025,T,07:50,17:00,07:36:28,17:21:00,NaN,NaN,NaN,NaN,Entrada y salida


In [23]:
#Olvido marcas -- Procesar reporte de marcas corregidas y reporte bruto combinado de rflex. 
#Filtrar y agrupar a las personas con olvido de marcas

#Reporte de marcas corregidas - Rflex
df_marcas_filtrado = df_marcas_filtrado[df_marcas_filtrado['Puesto'].isin(puestos_filtrados)]

df_marcas_filtrado['Fecha'] = pd.to_datetime(df_marcas_filtrado['Fecha'], format='%d-%m-%Y', errors='coerce')

df_marcas_filtrado['Periodo'] = df_marcas_filtrado['Fecha'].apply(
    lambda x: 'Periodo 1' if fecha_inicio_q1 <= x <= fecha_fin_q1 else (
        'Periodo 2' if fecha_inicio_q2 <= x <= fecha_fin_q2 else 'Fuera de rango'
    )
)

resumen_olvido_marcas = df_marcas_filtrado.groupby(['Rut', 'Nombre', 'Fecha', 'Periodo']).agg({
    'Motivo marca de entrada': 'count',
    'Motivo marca de salida' : 'count'
})  # Contar total de olvido de marcas

print("Olvido de marcas:")
print(f'Cantidad de personas encontrados = {resumen_olvido_marcas.shape[0]}')
#display(resumen_olvido_marcas)
display(resumen_olvido_marcas[resumen_olvido_marcas['Motivo marca de entrada'] != 0])

Olvido de marcas:
Cantidad de personas encontrados = 80


Motivo marca de entrada  \
Rut        Nombre                              Fecha      Periodo                              
10706013-8 Francisco Alejandro Soto Nuñez      2025-08-19 Periodo 1                        1   
                                               2025-08-20 Periodo 1                        1   
10875002-2 Ricardo Alejandro Muñoz Escobar     2025-08-19 Periodo 1                        1   
12474911-5 Marcelo Ricardo Salazar Alvarez     2025-08-13 Periodo 1                        1   
                                               2025-08-22 Periodo 2                        1   
12632762-5 Juan Francisco Torres Olguin        2025-08-11 Periodo 1                        1   
12634159-8 Rodrigo David Arenas Guerra         2025-08-08 Periodo 1                        1   
12857871-4 Sixto Manuel Beiza Mardones         2025-09-04 Periodo 2                        1   
13697731-8 Ricardo Antonio Silva Tapia         2025-08-13 Periodo 1                        1   
                                               2025-08-14 Periodo 1                        1   
14194403-7 Manuel Jesus Abarca Moreira         2025-09-01 Periodo 2                        1   
15464491-1 Juan Pablo Sanchez Montano          2025-08-21 Periodo 2                        1   
15473198-9 Cristian Fernando Barrera Navarrete 2025-08-21 Periodo 2                        1   
                                               2025-08-25 Periodo 2                        1   
17292954-0 Sergio Manuel Aros Lobos            2025-08-28 Periodo 2                        1   
17308379-3 Alexis  González Quezada            2025-08-14 Periodo 1                        1   
17781833-K Luis Jonathan Palma Diaz            2025-08-18 Periodo 1                        1   
18060825-7 Jonatan Rubén Díaz Lara             2025-08-28 Periodo 2                        1   
18116870-6 Edgar Habraham Rupallan Guaringa    2025-08-29 Periodo 2                        1   
18836244-3 Mirko Andres Godoy Rojas            2025-08-22 Periodo 2                        1   
18906244-3 Marco Elizer Rojas Mora             2025-09-04 Periodo 2                        1   
20118367-7 Benjamín Sebastián Bustos Flores    2025-08-20 Periodo 1                        1   
20761765-2 Alejandro Andrés Ayala Salas        2025-09-04 Periodo 2                        1   
22593492-4 Pedro Eduardo Triviño Ordoñez       2025-08-26 Periodo 2                        1   
25445747-7 Jhon Esteban Urbano Sanchez         2025-08-12 Periodo 1                        1   
                                               2025-08-22 Periodo 2                        1   
                                               2025-08-25 Periodo 2                        1   
27064835-5 Germán  Leyes Portilla              2025-09-04 Periodo 2                        1   
28798122-8 Hernán Gabriel Piterá Piterá        2025-08-25 Periodo 2                        1   

                                                                     Motivo marca de salida  
Rut        Nombre                              Fecha      Periodo                            
10706013-8 Francisco Alejandro Soto Nuñez      2025-08-19 Periodo 1                       0  
                                               2025-08-20 Periodo 1                       0  
10875002-2 Ricardo Alejandro Muñoz Escobar     2025-08-19 Periodo 1                       0  
12474911-5 Marcelo Ricardo Salazar Alvarez     2025-08-13 Periodo 1                       0  
                                               2025-08-22 Periodo 2                       0  
12632762-5 Juan Francisco Torres Olguin        2025-08-11 Periodo 1                       0  
12634159-8 Rodrigo David Arenas Guerra         2025-08-08 Periodo 1                       0  
12857871-4 Sixto Manuel Beiza Mardones         2025-09-04 Periodo 2                       0  
13697731-8 Ricardo Antonio Silva Tapia         2025-08-13 Periodo 1                       0  
                                               2025-08-14 

Desde aquí se busca en reporte bruto si hay omisiones de marca. Debe estar vacío si se corrigieron todas desde rflex. 

In [23]:
# #Filtro de df_atrasos (Reporte Rflex Bruto Combinado) para ver solo los casos sin permiso 
# #para buscar olvidos de marca

# columna_horas = df_atrasos_filtrado.columns[df_atrasos_filtrado.columns.str.contains('hora', case=False, na=False)]
# df_atrasos_filtrado['nombreTipoAusencia'] = df_atrasos_filtrado['nombreTipoAusencia'].str.replace(' ', 'Sin permiso')

# df_sin_permiso = df_atrasos_filtrado[df_atrasos_filtrado['nombreTipoAusencia'] == 'Sin permiso']

# display(df_sin_permiso[['nombreTipoAusencia', 'nombreCompleto', 'fecha'] + columna_horas.tolist()])

In [24]:
# #Reemplazar valores en blanco con "Omisión de marca"
# df_sin_permiso['HoraInicioMarcaReloj'] = df_sin_permiso['HoraInicioMarcaReloj'].str.replace(' ', 'Omisión de marca')
# df_sin_permiso['HoraTerminoMarcaReloj'] = df_sin_permiso['HoraTerminoMarcaReloj'].str.replace(' ', 'Omisión de marca')

In [25]:
# df_omisiones = df_sin_permiso[(df_sin_permiso['HoraInicioMarcaReloj'] == 'Omisión de marca') | (df_sin_permiso['HoraTerminoMarcaReloj'] == 'Omisión de marca')]
# display(df_omisiones)

# if df_omisiones.empty:
#     print("No hay omisiones de marca en el periodo por rflex. Lo que significa que todas fueron resultas en sistema. Guiarse por reporte de marcas corregidas")
# #No hay omisiones de marca en el periodo por rflex. Lo que significa que todas fueron resultas en sistema.#

Hasta el bloque de arriba se buscan omisiones desde el Reporte Bruto Combinado. Debe estar vacío si se corrigieron todas las marcas en rflex 

In [24]:
#Procesar atrasos desde reporte bruto combinado de rflex
import pandas as pd
columnas_necesarias = ['nombreTipoAusencia' ,'fecha', 'horaInicio', 'horaProgramadaInicio', 'HoraInicioMarcaReloj', 'horaTermino', 'horaProgramadaTermino', 'HoraTerminoMarcaReloj', 'rut', 'nombre', 'apellidoPaterno', 'apellidoMaterno', 'Centrocosto_trabajador', 'PuestoIntegracion_trabajador', 'nombreMotivoPago']
df_atrasos_calculo = pd.read_excel(r"C:\Users\bgacitua\Desktop\Repositorio_GitHub\Scripts de Python\Script Bono Asistencia\Archivos de trabajo\2025-09-12 08_30_21-2018778312-reporte_turnos_bruto_combinado.xlsx")

df_atrasos_calculo = df_atrasos_calculo[columnas_necesarias]
df_atrasos_calculo['horaInicio'] = df_atrasos_calculo['fecha'].astype(str).str.cat(df_atrasos_calculo['horaInicio'].astype(str), sep=' ')

display(df_atrasos_calculo.head())

,nombreTipoAusencia,fecha,horaInicio,horaProgramadaInicio,HoraInicioMarcaReloj,horaTermino,horaProgramadaTermino,HoraTerminoMarcaReloj,rut,nombre,apellidoPaterno,apellidoMaterno,Centrocosto_trabajador,PuestoIntegracion_trabajador,nombreMotivoPago
0,,2025-08-06,2025-08-06 07:50:00,2025-08-06 07:50:00,2025-08-06 07:50:00,17:00:00,2025-08-06 17:00:00,2025-08-06 17:00:00,69866409,Gloria,Gonzalez,Castro,Administración,Secretaria,
1,,2025-08-07,2025-08-07 07:50:00,2025-08-07 07:50:00,2025-08-07 07:02:01,17:00:00,2025-08-07 17:00:00,2025-08-07 16:57:16,69866409,Gloria,Gonzalez,Castro,Administración,Secretaria,
2,,2025-08-08,2025-08-08 07:50:00,2025-08-08 07:50:00,2025-08-08 07:30:44,15:30:00,2025-08-08 15:30:00,2025-08-08 15:30:00,69866409,Gloria,Gonzalez,Castro,Administración,Secretaria,
3,,2025-08-11,2025-08-11 07:50:00,2025-08-11 07:50:00,2025-08-11 07:23:45,17:00:00,2025-08-11 17:00:00,2025-08-11 17:01:52,69866409,Gloria,Gonzalez,Castro,Administración,Secretaria,
4,,2025-08-12,2025-08-12 07:50:00,2025-08-12 07:50:00,2025-08-12 07:50:00,17:00:00,2025-08-12 17:00:00,2025-08-12 16:55:48,69866409,Gloria,Gonzalez,Castro,Administración,Secretaria,


In [25]:
# Convertir las columnas a formato datetime, ignorando errores y valores vacíos
df_atrasos_calculo['horaInicio'] = pd.to_datetime(df_atrasos_calculo['horaInicio'], errors='coerce')
df_atrasos_calculo['HoraInicioMarcaReloj'] = pd.to_datetime(df_atrasos_calculo['HoraInicioMarcaReloj'], errors='coerce')
df_atrasos_calculo['horaTermino'] = pd.to_datetime(df_atrasos_calculo['horaTermino'], errors='coerce')
df_atrasos_calculo['HoraTerminoMarcaReloj'] = pd.to_datetime(df_atrasos_calculo['HoraTerminoMarcaReloj'], errors='coerce')

# Calcular la diferencia de tiempo para cada fila (en formato datetime)
df_atrasos_calculo['Diferencia'] = df_atrasos_calculo['horaInicio'] - df_atrasos_calculo['HoraInicioMarcaReloj']

# Extraer los minutos y segundos de la columna 'Diferencia'
df_atrasos_calculo['minutos_antes'] = df_atrasos_calculo['Diferencia'].dt.total_seconds() / 60
df_atrasos_calculo['segundos_antes'] = df_atrasos_calculo['Diferencia'].dt.total_seconds() % 60


C:\Users\bgacitua\AppData\Local\Temp\ipykernel_4648\3983933390.py:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_atrasos_calculo['horaTermino'] = pd.to_datetime(df_atrasos_calculo['horaTermino'], errors='coerce')


In [26]:
display(df_atrasos_calculo.head())

,nombreTipoAusencia,fecha,horaInicio,horaProgramadaInicio,HoraInicioMarcaReloj,horaTermino,horaProgramadaTermino,HoraTerminoMarcaReloj,rut,nombre,apellidoPaterno,apellidoMaterno,Centrocosto_trabajador,PuestoIntegracion_trabajador,nombreMotivoPago,Diferencia,minutos_antes,segundos_antes
0,,2025-08-06,2025-08-06 07:50:00,2025-08-06 07:50:00,2025-08-06 07:50:00,2025-09-12 17:00:00,2025-08-06 17:00:00,2025-08-06 17:00:00,69866409,Gloria,Gonzalez,Castro,Administración,Secretaria,,0 days 00:00:00,0.000000,0.0
1,,2025-08-07,2025-08-07 07:50:00,2025-08-07 07:50:00,2025-08-07 07:02:01,2025-09-12 17:00:00,2025-08-07 17:00:00,2025-08-07 16:57:16,69866409,Gloria,Gonzalez,Castro,Administración,Secretaria,,0 days 00:47:59,47.983333,59.0
2,,2025-08-08,2025-08-08 07:50:00,2025-08-08 07:50:00,2025-08-08 07:30:44,2025-09-12 15:30:00,2025-08-08 15:30:00,2025-08-08 15:30:00,69866409,Gloria,Gonzalez,Castro,Administración,Secretaria,,0 days 00:19:16,19.266667,16.0
3,,2025-08-11,2025-08-11 07:50:00,2025-08-11 07:50:00,2025-08-11 07:23:45,2025-09-12 17:00:00,2025-08-11 17:00:00,2025-08-11 17:01:52,69866409,Gloria,Gonzalez,Castro,Administración,Secretaria,,0 days 00:26:15,26.250000,15.0
4,,2025-08-12,2025-08-12 07:50:00,2025-08-12 07:50:00,2025-08-12 07:50:00,2025-09-12 17:00:00,2025-08-12 17:00:00,2025-08-12 16:55:48,69866409,Gloria,Gonzalez,Castro,Administración,Secretaria,,0 days 00:00:00,0.000000,0.0


In [27]:
df_atrasos_calculo['nombreCompleto'] = df_atrasos_calculo['nombre'].str.cat([
    df_atrasos_calculo['apellidoPaterno'], 
    df_atrasos_calculo['apellidoMaterno']], sep=' ', na_rep='')

df_atrasos_calculo = df_atrasos_calculo[df_atrasos_calculo['PuestoIntegracion_trabajador'].isin(puestos_filtrados)]
print(f"Filas originales: {df_atrasos_calculo.shape[0]}")
print(f"Filas después del filtro: {df_atrasos_calculo.shape[0]}")

columnas_finales = ['nombreCompleto', 'rut', 'PuestoIntegracion_trabajador', 'Centrocosto_trabajador', 'fecha', 'horaInicio', 'HoraInicioMarcaReloj', 'minutos_antes', 'segundos_antes', 'nombreTipoAusencia']
df_atrasos_calculo_filtrado = df_atrasos_calculo[columnas_finales]
permisos_aux = [' ']
horas_permisos_filtrados = df_atrasos_calculo_filtrado[df_atrasos_calculo_filtrado['nombreTipoAusencia'].isin(permisos_aux)]

horas_permisos_exportar = horas_permisos_filtrados[horas_permisos_filtrados['minutos_antes'] < -10]

horas_permisos_exportar['fecha'] = pd.to_datetime(horas_permisos_exportar['fecha'], errors='coerce')

horas_permisos_exportar['periodo'] = horas_permisos_exportar['fecha'].apply(
    lambda x: 'Periodo 1' if fecha_inicio_q1 <= x <= fecha_fin_q1 else (
        'Periodo 2' if fecha_inicio_q2 <= x <= fecha_fin_q2 else 'Fuera de rango'
    )
)

agrupado_permisos_exportar = horas_permisos_exportar.groupby(['nombreCompleto', 'rut', 'PuestoIntegracion_trabajador','periodo', 'Centrocosto_trabajador']).agg({
    'minutos_antes': 'count',
    'fecha': ['min', 'max']
}).reset_index()


#display(agrupado_permisos_exportar)
display(horas_permisos_exportar)


#ruta = r"C:\Users\bgacitua\Desktop\Respaldo marcas Morpho\horas_permisos_exportar.xlsx"
#horas_permisos_exportar.to_excel(ruta, index=False)


Filas originales: 5644
Filas después del filtro: 5644


C:\Users\bgacitua\AppData\Local\Temp\ipykernel_4648\3619606052.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  horas_permisos_exportar['fecha'] = pd.to_datetime(horas_permisos_exportar['fecha'], errors='coerce')
C:\Users\bgacitua\AppData\Local\Temp\ipykernel_4648\3619606052.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  horas_permisos_exportar['periodo'] = horas_permisos_exportar['fecha'].apply(


,nombreCompleto,rut,PuestoIntegracion_trabajador,Centrocosto_trabajador,fecha,horaInicio,HoraInicioMarcaReloj,minutos_antes,segundos_antes,nombreTipoAusencia,periodo
160,Marie Etienne,256248786,Asistente De Servicios Generales,Lavandería Producción,2025-09-02,2025-09-02 11:00:00,2025-09-02 11:11:04,-11.066667,56.0,,Periodo 2
251,Eyver Roa Vargas,261696436,Operario,Administración Gral De Producción,2025-08-11,2025-08-11 07:50:00,2025-08-11 08:01:05,-11.083333,55.0,,Periodo 1
262,Eyver Roa Vargas,261696436,Operario,Administración Gral De Producción,2025-08-27,2025-08-27 07:50:00,2025-08-27 08:18:06,-28.100000,54.0,,Periodo 2
264,Eyver Roa Vargas,261696436,Operario,Administración Gral De Producción,2025-08-29,2025-08-29 07:50:00,2025-08-29 08:08:25,-18.416667,35.0,,Periodo 2
991,Bastian Santander Cruces,203294352,Inspector De Proceso,Aseguramiento De Calidad,2025-08-19,2025-08-19 07:50:00,2025-08-19 10:42:03,-172.050000,57.0,,Periodo 1
...,...,...,...,...,...,...,...,...,...,...,...
10673,Juan González Soto,205808078,Asistente De Laboratorio,Desarrollo Sabores Bebidas,2025-08-27,2025-08-27 07:50:00,2025-08-27 08:15:07,-25.116667,53.0,,Periodo 2
10679,Juan González Soto,205808078,Asistente De Laboratorio,Desarrollo Sabores Bebidas,2025-09-04,2025-09-04 07:50:00,2025-09-04 08:07:20,-17.333333,40.0,,Periodo 2
10708,Benjamín Cornejo Cárdenas,205751696,Asistente De Laboratorio,Desarrollo Sabores Dulces,2025-08-11,2025-08-11 07:50:00,2025-08-11 11:11:15,-201.250000,45.0,,Periodo 1
10721,Benjamín Cornejo Cárdenas,205751696,Asistente De Laboratorio,Desarrollo Sabores Dulces,2025-08-29,2025-08-29 07:50:00,2025-08-29 08:01:25,-11.416667,35.0,,Periodo 2


UNIR RUT + PERIODO EN CADA DATAFRAME PARA LUEGO HACER COLUMNAS PARA CÁLCULO AUTOMÁTICO DEL BONO

DEL DF CON TODOS LOS TRABAJADORES, CREAR COLUMNAS PARA CADA ITEM QUE BUSQUEN Y CUENTEN POR RUT + PERIODO 

LUEGO, DESARROLLAR COLUMNA CON LOGICA PARA QUE ASIGNE AUTOMÁTICAMENTE EL BONO

In [28]:
#Modificar columnas de rut para dejarlos sin carácteres especiales

#horas_permisos_exportar = listo
#df_atrasos_filtrado_permisos = listo
''''
df_marcas_filtrado.info()
'''
df_marcas_filtrado['Rut'] = df_marcas_filtrado['Rut'].str.replace('.', '', regex=False).str.replace('-', '', regex=False)
df_marcas_filtrado['Rut'] = df_marcas_filtrado['Rut'].astype(str).str.upper()
df_marcas_filtrado = df_marcas_filtrado.rename(columns={'Rut': 'rut'})
#df_marcas_filtrado = listo

#display(df_marcas_filtrado.head())


In [29]:
#Concatenar rut + columna de periodo para hacer merge único
horas_permisos_exportar['Busqueda'] = horas_permisos_exportar['rut'].astype(str).str.upper() + '_' + horas_permisos_exportar['periodo'].astype(str)
columnas_ordenadas_horas_exportar = ['Busqueda', 'nombreCompleto', 'rut', 'PuestoIntegracion_trabajador', 'fecha', 
                                    'horaInicio', 'HoraInicioMarcaReloj', 'minutos_antes',
                                    'nombreTipoAusencia', 'periodo']
horas_permisos_exportar = horas_permisos_exportar[columnas_ordenadas_horas_exportar]
horas_permisos_exportar = horas_permisos_exportar[horas_permisos_exportar['minutos_antes'] < -10]

display(horas_permisos_exportar.head())

C:\Users\bgacitua\AppData\Local\Temp\ipykernel_4648\3047790997.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  horas_permisos_exportar['Busqueda'] = horas_permisos_exportar['rut'].astype(str).str.upper() + '_' + horas_permisos_exportar['periodo'].astype(str)


,Busqueda,nombreCompleto,rut,PuestoIntegracion_trabajador,fecha,horaInicio,HoraInicioMarcaReloj,minutos_antes,nombreTipoAusencia,periodo
160,256248786_Periodo 2,Marie Etienne,256248786,Asistente De Servicios Generales,2025-09-02,2025-09-02 11:00:00,2025-09-02 11:11:04,-11.066667,,Periodo 2
251,261696436_Periodo 1,Eyver Roa Vargas,261696436,Operario,2025-08-11,2025-08-11 07:50:00,2025-08-11 08:01:05,-11.083333,,Periodo 1
262,261696436_Periodo 2,Eyver Roa Vargas,261696436,Operario,2025-08-27,2025-08-27 07:50:00,2025-08-27 08:18:06,-28.100000,,Periodo 2
264,261696436_Periodo 2,Eyver Roa Vargas,261696436,Operario,2025-08-29,2025-08-29 07:50:00,2025-08-29 08:08:25,-18.416667,,Periodo 2
991,203294352_Periodo 1,Bastian Santander Cruces,203294352,Inspector De Proceso,2025-08-19,2025-08-19 07:50:00,2025-08-19 10:42:03,-172.050000,,Periodo 1


In [30]:
#Unificar el nombre de permisos y licencias para agruparlos en categorías
df_atrasos_filtrado_permisos['Busqueda'] = df_atrasos_filtrado_permisos['rut'].astype(str).str.upper() + '_' + df_atrasos_filtrado_permisos['periodo'].astype(str)
columnas_ordenadas_atrasos_permisos = ['Busqueda', 'nombreTipoAusencia', 'fecha', 'horaInicio', 'horaProgramadaInicio', 'HoraInicioMarcaReloj',
                                        'horaTermino', 'horaProgramadaTermino', 'HoraTerminoMarcaReloj', 'rut', 'nombre', 'apellidoPaterno', 'apellidoMaterno',
                                        'Centrocosto_trabajador', 'PuestoIntegracion_trabajador', 'nombreMotivoPago', 'periodo']
df_atrasos_filtrado_permisos = df_atrasos_filtrado_permisos[columnas_ordenadas_atrasos_permisos]

#Limpiar tipos de permisos para agruparlos en categorías
df_atrasos_filtrado_permisos['nombreTipoAusencia'] = df_atrasos_filtrado_permisos['nombreTipoAusencia'].str.replace(
    'Licencia postnatal', 'Licencia', regex=True)
df_atrasos_filtrado_permisos['nombreTipoAusencia'] = df_atrasos_filtrado_permisos['nombreTipoAusencia'].str.replace(
    'Licencia Accidente Trayecto', 'Licencia', regex=True)
df_atrasos_filtrado_permisos['nombreTipoAusencia'] = df_atrasos_filtrado_permisos['nombreTipoAusencia'].str.replace(
    'Licencia Accidente Trabajo', 'Licencia', regex=True)
df_atrasos_filtrado_permisos['nombreTipoAusencia'] = df_atrasos_filtrado_permisos['nombreTipoAusencia'].str.replace(
    'Licencia Accidente', 'Licencia', regex=True)     
df_atrasos_filtrado_permisos['nombreTipoAusencia'] = df_atrasos_filtrado_permisos['nombreTipoAusencia'].str.replace(
    'Permiso sin goce de sueldo', 'Permiso', regex=True) 
df_atrasos_filtrado_permisos['nombreTipoAusencia'] = df_atrasos_filtrado_permisos['nombreTipoAusencia'].str.replace(
    'Permiso con goce de sueldo', 'Permiso', regex=True) 

df_atrasos_filtrado_permisos['Busqueda'] = df_atrasos_filtrado_permisos['Busqueda'].astype(str) + '_' + df_atrasos_filtrado_permisos['nombreTipoAusencia'].astype(str)

display(df_atrasos_filtrado_permisos['nombreTipoAusencia'].unique())
display(df_atrasos_filtrado_permisos.head())

array(['Licencia', 'Permiso', 'Inasistencia'], dtype=object)

,Busqueda,nombreTipoAusencia,fecha,horaInicio,horaProgramadaInicio,HoraInicioMarcaReloj,horaTermino,horaProgramadaTermino,HoraTerminoMarcaReloj,rut,nombre,apellidoPaterno,apellidoMaterno,Centrocosto_trabajador,PuestoIntegracion_trabajador,nombreMotivoPago,periodo
955,158929724_Periodo 1_Licencia,Licencia,2025-08-06,07:50:00,07:50:00,,17:00:00,17:00:00,,158929724,Maritza,Marifil,Bustamante,Aseguramiento De Calidad,Inspector De Proceso,,Periodo 1
956,158929724_Periodo 1_Licencia,Licencia,2025-08-07,07:50:00,07:50:00,,17:00:00,17:00:00,,158929724,Maritza,Marifil,Bustamante,Aseguramiento De Calidad,Inspector De Proceso,,Periodo 1
957,158929724_Periodo 1_Licencia,Licencia,2025-08-08,07:50:00,07:50:00,,14:30:00,14:30:00,,158929724,Maritza,Marifil,Bustamante,Aseguramiento De Calidad,Inspector De Proceso,,Periodo 1
958,158929724_Periodo 1_Licencia,Licencia,2025-08-11,07:50:00,07:50:00,,17:00:00,17:00:00,,158929724,Maritza,Marifil,Bustamante,Aseguramiento De Calidad,Inspector De Proceso,,Periodo 1
959,158929724_Periodo 1_Licencia,Licencia,2025-08-12,07:50:00,07:50:00,,17:00:00,17:00:00,,158929724,Maritza,Marifil,Bustamante,Aseguramiento De Calidad,Inspector De Proceso,,Periodo 1


In [31]:
df_marcas_filtrado['Busqueda'] = df_marcas_filtrado['rut'].astype(str).str.upper() + '_' + df_marcas_filtrado['Periodo'].astype(str)
columnas_ordenadas_marcas = ['Busqueda','rut', 'Nombre', 'Unidad', 'Puesto', 'Fecha', 'Tipo de turno', 
                            'Entrada programada', 'Salida programada', 'Marca de entrada',
                            'Marca de salida', 'Motivo marca de entrada',
                            'Comentario marca de entrada', 'Motivo marca de salida',
                            'Comentario marca de salida', 'Tipo de corrección', 'Periodo']
df_marcas_filtrado = df_marcas_filtrado[columnas_ordenadas_marcas]
display(df_marcas_filtrado.head(5))

,Busqueda,rut,Nombre,Unidad,Puesto,Fecha,Tipo de turno,Entrada programada,Salida programada,Marca de entrada,Marca de salida,Motivo marca de entrada,Comentario marca de entrada,Motivo marca de salida,Comentario marca de salida,Tipo de corrección,Periodo
9,16872683K_Periodo 1,16872683K,Mario Rodolfo Orellana Fuentes,Planta Mezclado y Secado,Operario,2025-08-06,N4,22:00,06:00,21:39:18,06:00:00 *,NaN,NaN,Olvido de Marcaje,Olvido de marca,Solo salida,Periodo 1
58,154297715_Periodo 1,154297715,Manuel Alejandro Fuentes Yañez,Planta Mezclado y Secado,Operario,2025-08-06,T,07:50,17:00,07:49:45,17:00:00 *,NaN,NaN,Olvido de Marcaje,NaN,Solo salida,Periodo 1
59,154297715_Periodo 2,154297715,Manuel Alejandro Fuentes Yañez,Planta Mezclado y Secado,Operario,2025-09-02,T,07:50,17:00,07:50:00,16:54:54,NaN,NaN,NaN,NaN,Entrada y salida,Periodo 2
98,207290149_Periodo 1,207290149,Javier Alonso Alarcon Andrade,"Bodega Recepción, Despacho y Transporte",Ayudante De Bodega,2025-08-06,T,07:50,17:00,07:46:00,16:53:38,NaN,NaN,NaN,NaN,Entrada y salida,Periodo 1
101,118379756_Periodo 1,118379756,Sergio Vladimir Jerez Cea,Esencias y Env. Sabores,Operario,2025-08-06,T,07:50,17:00,07:43:37,17:00:00 *,NaN,NaN,Olvido de Marcaje,NaN,Solo salida,Periodo 1


In [32]:
#Agregar columnas para búsqueda en df_resultado
df_resultado['rut'] = df_resultado['rut'].str.replace('.', '', regex=False).str.replace('-', '', regex=False)
df_resultado['rut'] = df_resultado['rut'].astype(str).str.upper()

df_resultado['Busqueda Licencia 1'] = df_resultado['rut'].astype(str).str.upper() + '_Periodo 1' + '_Licencia'
df_resultado['Busqueda Licencia 2'] = df_resultado['rut'].astype(str).str.upper() + '_Periodo 2' + '_Licencia'

df_resultado['Busqueda Permiso 1'] = df_resultado['rut'].astype(str).str.upper() + '_Periodo 1' + '_Permiso'
df_resultado['Busqueda Permiso 2'] = df_resultado['rut'].astype(str).str.upper() + '_Periodo 2' + '_Permiso'

df_resultado['Busqueda Inasistencia 1'] = df_resultado['rut'].astype(str).str.upper() + '_Periodo 1' + '_Inasistencia'
df_resultado['Busqueda Inasistencia 2'] = df_resultado['rut'].astype(str).str.upper() + '_Periodo 2' + '_Inasistencia'

df_resultado['Busqueda General 1'] = df_resultado['rut'].astype(str).str.upper() + '_Periodo 1'
df_resultado['Busqueda General 2'] = df_resultado['rut'].astype(str).str.upper() + '_Periodo 2'

display(df_resultado.head(1))

,full_name,rut,active_since,status,name_role,cost_center,area_name,Busqueda Licencia 1,Busqueda Licencia 2,Busqueda Permiso 1,Busqueda Permiso 2,Busqueda Inasistencia 1,Busqueda Inasistencia 2,Busqueda General 1,Busqueda General 2
0,Víctor Antonio Riquelme Ulloa,140902152,2025-09-08,activo,Operario,6005,CARLOS CRAMER PRODUCTOS AROMÁTICOS S.A. C.I.,140902152_Periodo 1_Licencia,140902152_Periodo 2_Licencia,140902152_Periodo 1_Permiso,140902152_Periodo 2_Permiso,140902152_Periodo 1_Inasistencia,140902152_Periodo 2_Inasistencia,140902152_Periodo 1,140902152_Periodo 2


In [33]:
#Busquedas de permisos
cuenta_licencias_p1 = df_atrasos_filtrado_permisos['Busqueda'].value_counts()
cuenta_licencias_p2 = df_atrasos_filtrado_permisos['Busqueda'].value_counts()
cuenta_permisos_p1 = df_atrasos_filtrado_permisos['Busqueda'].value_counts()
cuenta_permisos_p2 = df_atrasos_filtrado_permisos['Busqueda'].value_counts()
cuenta_inasistencias_p1 = df_atrasos_filtrado_permisos['Busqueda'].value_counts()
cuenta_inasistencias_p2 = df_atrasos_filtrado_permisos['Busqueda'].value_counts()

df_resultado['Licencias Periodo 1'] = df_resultado['Busqueda Licencia 1'].map(cuenta_licencias_p1).fillna(0).astype(int)
df_resultado['Licencias Periodo 2'] = df_resultado['Busqueda Licencia 2'].map(cuenta_licencias_p2).fillna(0).astype(int)
df_resultado['Permisos Periodo 1'] = df_resultado['Busqueda Permiso 1'].map(cuenta_permisos_p1).fillna(0).astype(int)
df_resultado['Permisos Periodo 2'] = df_resultado['Busqueda Permiso 2'].map(cuenta_permisos_p2).fillna(0).astype(int)
df_resultado['Inasistencias Periodo 1'] = df_resultado['Busqueda Inasistencia 1'].map(cuenta_inasistencias_p1).fillna(0).astype(int)
df_resultado['Inasistencias Periodo 2'] = df_resultado['Busqueda Inasistencia 2'].map(cuenta_inasistencias_p2).fillna(0).astype(int)

#Busqueda de atrasos
cuenta_atrasos_p1 = horas_permisos_exportar['Busqueda'].value_counts()
cuenta_atrasos_p2 = horas_permisos_exportar['Busqueda'].value_counts()
df_resultado['Atrasos Periodo 1'] = df_resultado['Busqueda General 1'].map(cuenta_atrasos_p1).fillna(0).astype(int)
df_resultado['Atrasos Periodo 2'] = df_resultado['Busqueda General 2'].map(cuenta_atrasos_p2).fillna(0).astype(int)

#Busqueda de olvido de marcas
cuenta_olvido_marcas_p1 = df_marcas_filtrado['Busqueda'].value_counts()
cuenta_olvido_marcas_p2 = df_marcas_filtrado['Busqueda'].value_counts()
df_resultado['Olvido Marca Periodo 1'] = df_resultado['Busqueda General 1'].map(cuenta_olvido_marcas_p1).fillna(0).astype(int)
df_resultado['Olvido Marca Periodo 2'] = df_resultado['Busqueda General 2'].map(cuenta_olvido_marcas_p2).fillna(0).astype(int)

#display(df_resultado[df_resultado['Licencias Periodo 1'] > 0])

#Aplanar columnas
columnas_reordenadas_df_resultado = ['full_name', 'rut', 'active_since', 'status', 'name_role', 'cost_center', 'area_name', 'Licencias Periodo 1', 
                                    'Licencias Periodo 2', 'Permisos Periodo 1', 'Permisos Periodo 2', 'Inasistencias Periodo 1', 
                                    'Inasistencias Periodo 2', 'Atrasos Periodo 1', 'Atrasos Periodo 2',
                                    'Olvido Marca Periodo 1', 'Olvido Marca Periodo 2']

df_final = df_resultado[columnas_reordenadas_df_resultado]
display(df_final.head(1))

,full_name,rut,active_since,status,name_role,cost_center,area_name,Licencias Periodo 1,Licencias Periodo 2,Permisos Periodo 1,Permisos Periodo 2,Inasistencias Periodo 1,Inasistencias Periodo 2,Atrasos Periodo 1,Atrasos Periodo 2,Olvido Marca Periodo 1,Olvido Marca Periodo 2
0,Víctor Antonio Riquelme Ulloa,140902152,2025-09-08,activo,Operario,6005,CARLOS CRAMER PRODUCTOS AROMÁTICOS S.A. C.I.,0,0,0,0,0,0,0,0,0,0


In [34]:
df_resultado.columns

Index(['full_name', 'rut', 'active_since', 'status', 'name_role',
       'cost_center', 'area_name', 'Busqueda Licencia 1',
       'Busqueda Licencia 2', 'Busqueda Permiso 1', 'Busqueda Permiso 2',
       'Busqueda Inasistencia 1', 'Busqueda Inasistencia 2',
       'Busqueda General 1', 'Busqueda General 2', 'Licencias Periodo 1',
       'Licencias Periodo 2', 'Permisos Periodo 1', 'Permisos Periodo 2',
       'Inasistencias Periodo 1', 'Inasistencias Periodo 2',
       'Atrasos Periodo 1', 'Atrasos Periodo 2', 'Olvido Marca Periodo 1',
       'Olvido Marca Periodo 2'],
      dtype='object')

In [35]:
columnas_periodo_1 = df_final.columns[df_final.columns.str.contains('Periodo 1', case=False, na=False)].tolist() + ['active_since']
columnas_periodo_2 = df_final.columns[df_final.columns.str.contains('Periodo 2', case=False, na=False)].tolist() + ['active_since']

display(columnas_periodo_1)
display(columnas_periodo_2)

['Licencias Periodo 1',
 'Permisos Periodo 1',
 'Inasistencias Periodo 1',
 'Atrasos Periodo 1',
 'Olvido Marca Periodo 1',
 'active_since']

['Licencias Periodo 2',
 'Permisos Periodo 2',
 'Inasistencias Periodo 2',
 'Atrasos Periodo 2',
 'Olvido Marca Periodo 2',
 'active_since']

In [36]:
df_final_con_bono = df_final.copy()

def calcular_bono_p1(row):
    #Transformar col6 a datetime
    row[columnas_periodo_1[5]] = pd.to_datetime(row[columnas_periodo_1[5]], errors='coerce')

    # Suma de las primeras tres columnas del periodo 1
    suma_3 = row[columnas_periodo_1[0]] + row[columnas_periodo_1[1]] + row[columnas_periodo_1[2]]
    col4 = row[columnas_periodo_1[3]]
    col5 = row[columnas_periodo_1[4]]
    col6 = row[columnas_periodo_1[5]]
    
    if suma_3 >= 1:
        return "0"
    elif col4 >= 3:
        return "0"
    elif col5 >= 3:
        return "0"
    elif col6 > fecha_inicio_q2:
        return "0"
    else:
        return "0.5"

df_final_con_bono['Bono Asistencia P1'] = df_final_con_bono.apply(calcular_bono_p1, axis=1)

def calcular_bono_p2(row):
    #Transformar col6 a datetime
    row[columnas_periodo_2[5]] = pd.to_datetime(row[columnas_periodo_2[5]], errors='coerce')

    # Suma de las primeras tres columnas del periodo 1
    suma_3 = row[columnas_periodo_2[0]] + row[columnas_periodo_2[1]] + row[columnas_periodo_2[2]]
    col4 = row[columnas_periodo_2[3]]
    col5 = row[columnas_periodo_2[4]]
    col6 = row[columnas_periodo_2[5]]
    
    if suma_3 >= 1:
        return "0"
    elif col4 >= 3:
        return "0"
    elif col5 >= 3:
        return "0"
    elif col6 > fecha_inicio_q2:
        return "0"
    else:
        return "0.5"

df_final_con_bono['Bono Asistencia P2'] = df_final_con_bono.apply(calcular_bono_p2, axis=1)

df_final_con_bono['Bono Total'] = df_final_con_bono['Bono Asistencia P1'].astype(float) + df_final_con_bono['Bono Asistencia P2'].astype(float)

df_final_con_bono.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 282 entries, 0 to 281
Data columns (total 20 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   full_name                282 non-null    object 
 1   rut                      282 non-null    object 
 2   active_since             282 non-null    object 
 3   status                   282 non-null    object 
 4   name_role                282 non-null    object 
 5   cost_center              282 non-null    object 
 6   area_name                282 non-null    object 
 7   Licencias Periodo 1      282 non-null    int64  
 8   Licencias Periodo 2      282 non-null    int64  
 9   Permisos Periodo 1       282 non-null    int64  
 10  Permisos Periodo 2       282 non-null    int64  
 11  Inasistencias Periodo 1  282 non-null    int64  
 12  Inasistencias Periodo 2  282 non-null    int64  
 13  Atrasos Periodo 1        282 non-null    int64  
 14  Atrasos Periodo 2        2

In [37]:
display(df_final_con_bono.head(5))

,full_name,rut,active_since,status,name_role,cost_center,area_name,Licencias Periodo 1,Licencias Periodo 2,Permisos Periodo 1,Permisos Periodo 2,Inasistencias Periodo 1,Inasistencias Periodo 2,Atrasos Periodo 1,Atrasos Periodo 2,Olvido Marca Periodo 1,Olvido Marca Periodo 2,Bono Asistencia P1,Bono Asistencia P2,Bono Total
0,Víctor Antonio Riquelme Ulloa,140902152,2025-09-08,activo,Operario,6005,CARLOS CRAMER PRODUCTOS AROMÁTICOS S.A. C.I.,0,0,0,0,0,0,0,0,0,0,0,0,0.0
1,Luis Andrés Rojas Romero,16459775K,2025-09-08,activo,Operario,6005,CARLOS CRAMER PRODUCTOS AROMÁTICOS S.A. C.I.,0,0,0,0,0,0,0,0,0,0,0,0,0.0
2,Araceli Alejandra Berríos Vera,199163248,2025-09-08,activo,Inspector De Proceso,6106,CARLOS CRAMER PRODUCTOS AROMÁTICOS S.A. C.I.,0,0,0,0,0,0,0,0,0,0,0,0,0.0
3,Kevin Hernán Zamora Flores,204194211,2025-09-08,activo,Operario,6005,CARLOS CRAMER PRODUCTOS AROMÁTICOS S.A. C.I.,0,0,0,0,0,0,0,0,0,0,0,0,0.0
4,Fernando Alonso Vera Clavero,221931599,2025-09-01,activo,Peoneta,6205,Servicios De Producción Y Logística Ccpa Ltda.,0,0,0,0,0,0,0,1,0,0,0,0,0.0


In [38]:
df_final_con_bono['rut'] = df_final_con_bono['rut'].str[:-1] + "-" + df_final_con_bono['rut'].str[-1]
display(df_final_con_bono.head(10))

,full_name,rut,active_since,status,name_role,cost_center,area_name,Licencias Periodo 1,Licencias Periodo 2,Permisos Periodo 1,Permisos Periodo 2,Inasistencias Periodo 1,Inasistencias Periodo 2,Atrasos Periodo 1,Atrasos Periodo 2,Olvido Marca Periodo 1,Olvido Marca Periodo 2,Bono Asistencia P1,Bono Asistencia P2,Bono Total
0,Víctor Antonio Riquelme Ulloa,14090215-2,2025-09-08,activo,Operario,6005,CARLOS CRAMER PRODUCTOS AROMÁTICOS S.A. C.I.,0,0,0,0,0,0,0,0,0,0,0,0,0.0
1,Luis Andrés Rojas Romero,16459775-K,2025-09-08,activo,Operario,6005,CARLOS CRAMER PRODUCTOS AROMÁTICOS S.A. C.I.,0,0,0,0,0,0,0,0,0,0,0,0,0.0
2,Araceli Alejandra Berríos Vera,19916324-8,2025-09-08,activo,Inspector De Proceso,6106,CARLOS CRAMER PRODUCTOS AROMÁTICOS S.A. C.I.,0,0,0,0,0,0,0,0,0,0,0,0,0.0
3,Kevin Hernán Zamora Flores,20419421-1,2025-09-08,activo,Operario,6005,CARLOS CRAMER PRODUCTOS AROMÁTICOS S.A. C.I.,0,0,0,0,0,0,0,0,0,0,0,0,0.0
4,Fernando Alonso Vera Clavero,22193159-9,2025-09-01,activo,Peoneta,6205,Servicios De Producción Y Logística Ccpa Ltda.,0,0,0,0,0,0,0,1,0,0,0,0,0.0
5,José Luis Cáceres Sandoval,12250916-8,2025-08-25,activo,Operario Almacenamiento y Gestión de Residuos,6103,CARLOS CRAMER PRODUCTOS AROMÁTICOS S.A. C.I.,0,0,0,0,0,0,0,1,0,0,0,0,0.0
6,Marcelo Del Carmen González Baeza,14485044-0,2025-08-25,activo,Operario,6005,CARLOS CRAMER PRODUCTOS AROMÁTICOS S.A. C.I.,0,0,0,0,0,0,0,1,0,0,0,0,0.0
7,Carolina Alejandra Tapia Burboa,17835585-6,2025-08-25,activo,Asistente De Laboratorio,3001,CARLOS CRAMER PRODUCTOS AROMÁTICOS S.A. C.I.,0,0,0,0,0,0,0,0,0,0,0,0,0.0
8,Jesús Damián Sarabia Gomez,19233137-4,2025-08-25,activo,Asistente De Laboratorio,3001,CARLOS CRAMER PRODUCTOS AROMÁTICOS S.A. C.I.,0,0,0,0,0,0,0,0,0,0,0,0,0.0
9,Luis Alfredo Escobar Pérez,19833080-9,2025-08-25,activo,Operario,6003,CARLOS CRAMER PRODUCTOS AROMÁTICOS S.A. C.I.,0,0,0,0,0,0,0,1,0,0,0,0,0.0


In [39]:
#Exportar archivo para Juan Cárcamo

import os
from datetime import datetime

# Usar la fecha actual o una fecha específica
fecha_actual = datetime.now()

# nombre_carpeta = f"Reporte Bono Asistencia {fecha_actual.strftime('%Y-%B')}"
# ruta_base = r"C:\Users\bgacitua\Desktop\Repositorio_GitHub\Scripts de Python"
# ruta_carpeta = os.path.join(ruta_base, nombre_carpeta)

# # Crear la carpeta si no existe
# os.makedirs(ruta_carpeta, exist_ok=True)

# # Guardar un DataFrame en esa carpeta
# ruta_archivo = ruta_archivo = os.path.join(
#     ruta_carpeta,
#     f"Reporte_Bono_Asistencia_Juan_Carcamo_{fecha_actual.strftime('%B')}.xlsx"
# )

# with pd.ExcelWriter(ruta_archivo) as writer:
#     resumen_atrasos_filtrado_permisos.to_excel(writer, sheet_name='Permisos Rflex')
#     resumen_olvido_marcas.to_excel(writer, sheet_name='Olvido marcas Rflex')
#     horas_permisos_exportar.to_excel(writer, sheet_name='Detalle Atrasos >10 min Rflex')

In [ ]:
#Exportar archivo con base de pago y bono asistencia
fecha_actual = datetime.now()

nombre_carpeta = f"Reporte Bono Asistencia {fecha_actual.strftime('%Y-%B')}"
ruta_base = r"C:\Users\bgacitua\Desktop\Repositorio_GitHub\Scripts de Python"
ruta_carpeta = os.path.join(ruta_base, nombre_carpeta)

# Crear la carpeta si no existe
os.makedirs(ruta_carpeta, exist_ok=True)

# Guardar un DataFrame en esa carpeta
ruta_archivo = ruta_archivo = os.path.join(
    ruta_carpeta,
    f"Bono_asistencia_generado_automáticamente_{fecha_actual.strftime('%B')}.xlsx"
)

with pd.ExcelWriter(ruta_archivo) as writer:
    df_final_con_bono.to_excel(writer, sheet_name='Base de pago')
